<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
import pickle

def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_list = pickle.load(open_file)
    open_file.close()
    return loaded_list

In [5]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [6]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [29]:
def calc_results(model, train_scaled, val_scaled, test_scaled, train_target, val_target, test_target,
                 test_scaled1, test_scaled2, test_scaled3, test_target1, test_target2, test_target3):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbmgs.best_score_ 
    result_dict['best_index'] = lgbmgs.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['precision'] = precision_score(test_target, y_predict)
    result_dict['recall'] = recall_score(test_target, y_predict)
    result_dict['f1score'] = f1_score(test_target, y_predict)
    result_dict['roc'] = roc_auc_score(test_target, y_predict)
    cm = confusion_matrix(test_target, y_predict)
    result_dict['tn'] = cm[0,0]
    result_dict['fp'] = cm[0,1]
    result_dict['fn'] = cm[1,0]
    result_dict['tp'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled1)
    result_dict['acc_test1'] = model.score(test_scaled1, test_target1)
    result_dict['precision1'] = precision_score(test_target1, y_predict)
    result_dict['recall1'] = recall_score(test_target1, y_predict)
    result_dict['f1score1'] = f1_score(test_target1, y_predict)
    result_dict['roc1'] = roc_auc_score(test_target1, y_predict)
    cm = confusion_matrix(test_target1, y_predict)
    result_dict['tn1'] = cm[0,0]
    result_dict['fp1'] = cm[0,1]
    result_dict['fn1'] = cm[1,0]
    result_dict['tp1'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled2)
    result_dict['acc_test2'] = model.score(test_scaled2, test_target2)
    result_dict['precision2'] = precision_score(test_target2, y_predict)
    result_dict['recall2'] = recall_score(test_target2, y_predict)
    result_dict['f1score2'] = f1_score(test_target2, y_predict)
    result_dict['roc2'] = roc_auc_score(test_target2, y_predict)
    cm = confusion_matrix(test_target2, y_predict)
    result_dict['tn2'] = cm[0,0]
    result_dict['fp2'] = cm[0,1]
    result_dict['fn2'] = cm[1,0]
    result_dict['tp2'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled3)
    result_dict['acc_test3'] = model.score(test_scaled3, test_target3)
    result_dict['precision3'] = precision_score(test_target3, y_predict)
    result_dict['recall3'] = recall_score(test_target3, y_predict)
    result_dict['f1score3'] = f1_score(test_target3, y_predict)
    result_dict['roc3'] = roc_auc_score(test_target3, y_predict)
    cm = confusion_matrix(test_target3, y_predict)
    result_dict['tn3'] = cm[0,0]
    result_dict['fp3'] = cm[0,1]
    result_dict['fn3'] = cm[1,0]
    result_dict['tp3'] = cm[1,1]
    return result_dict

In [30]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [96]:
new_col = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1',
       'pension_1', 'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1',
       'corporateetc_1', 'foreigneretc_1', 'dji_cr', 'dji_f_cr', 'ixic_cr',
       'ixic_f_cr', 'spx_cr', 'spx_f_cr', 'bond_kor_10_cr', 'bond_kor_2_cr',
       'dxy_cr', 'bond_usa_10_cr', 'bond_usa_2_cr', 'bond_usa_3m_cr',
       'kosdaq_cr', 'kospi_cr', 'krw_cr', 'sox_cr', 'vix_cr', 'wti_cr',
       'open_1', 'high_1', 'low_1', 'close_1', 'vol_1', 'retail_2',
       'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2',
       'pension_2', 'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2',
       'corporateetc_2', 'foreigneretc_2', 'dji_cr_2', 'dji_f_cr_2',
       'ixic_cr_2', 'ixic_f_cr_2', 'spx_cr_2', 'spx_f_cr_2',
       'bond_kor_10_cr_2', 'bond_kor_2_cr_2', 'dxy_cr_2', 'bond_usa_10_cr_2',
       'bond_usa_2_cr_2', 'bond_usa_3m_cr_2', 'kosdaq_cr_2', 'kospi_cr_2',
       'krw_cr_2', 'sox_cr_2', 'vix_cr_2', 'wti_cr_2', 'open_2', 'high_2',
       'low_2', 'close_2', 'vol_2', 'weekday', 'cr_00', 'cr_05', 'cr_10',
       'cr_15', 'cr_20']

In [217]:
com_name = 'sec'
fname = f'df_{com_name}_sel.pkl'
directory_for_ml = '../data/data_for_ml/expand_date/'

# fname = f'df_shgroup_sel.pkl'
# stock_name = 'shgroup'
directory_for_ml = '../data/data_for_ml/expand_date/'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name) 

# bank, financeetc는 결측치가 많아서 사용하지 않음.
# df.drop(['bank_1', 'bank_2', 'financeetc_1', 'financeetc_2'], axis=1, inplace=True)   

df = df[new_col]

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

test_interval = int((len(df) - split_n)/3)
data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -4]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -4]
test_input1 = df.iloc[split_n:split_n+test_interval, :-5]
test_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]
test_input3 = df.iloc[split_n+test_interval*2:, :-5]
test_target1 = df.iloc[split_n:split_n+test_interval, -4]
test_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -4]
test_target3 = df.iloc[split_n+test_interval*2:, -4]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)
test_scaled1 = scaler.transform(test_input1)
test_scaled2 = scaler.transform(test_input2)
test_scaled3 = scaler.transform(test_input3)

In [218]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')

In [219]:
param_base = {'boosting_type' : ['gbdt'], # ['gbdt', dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
              'num_leaves' :  [4, 5, 6, 7], # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
              'learning_rate' :  [0.003, 0.004, 0.005, 0.006],
#               'max_delta_step' :  [0.4, 0.5, 0.6],
              'n_estimators' :  [150, 200, 300, 400],
              'colsample_bytree' :  [0.7], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
              'subsample' :  [0.7],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
              'max_depth' :  [-1], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
              'objective': ['binary'],
              'metric': ['binary_logloss'],
              'scale_pos_weight': [1.5, 2, 2.5, 3.0], # posiive  증가, class imbalance 경감, scale_pos_weight > 0.0, default=1.0. 
              }

In [220]:
params_o = param_base.copy()

In [223]:
# directory가 없으면 만드는 과정
if not os.path.exists(com_name):
    os.makedirs(com_name)
    
iter = 4
while True:
    iter = iter + 1

    lgbm = None
    lgbmgs = None

    lgbm = lightgbm.LGBMClassifier(random_state=42)

    lgbmgs = GridSearchCV(estimator = lgbm,
                          param_grid = params_o,
                          cv = 7, # StratifiedKFold us default for binary or multiclass
                          scoring = 'precision', 
#                           scoring = 'accuracy', 
#                           scoring = ['accuracy', 'precision'], # refit 사용해야 함. 고로 사용하지 않음.
                          error_score='raise',
                          verbose = 1,
                          n_jobs=-1, # 자동 검색 적용
                         )
                          
    print("*** after lgbmgs ******")
    lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss') 
#     lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss', eval_set = (val_scaled, val_target)) 
    # eval_set가 있어야 "early_stopping_rounds"를 사용할 수 있음.

# save model
    stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
    dt = re.sub(r'[-:T]', '', stamp[5:16])
    joblib.dump(lgbmgs, f'./{com_name}/lgbm_v{iter}_{dt}.pkl') # gridsearchcv 저장
    joblib.dump(scaler, f'./{com_name}/scaler_v{iter}_{dt}.pkl') # scaler 저장
    
    df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), iter)
    result_dict = calc_results(lgbmgs.best_estimator_, 
                               train_scaled, val_scaled, test_scaled,  
                               train_target, val_target, test_target,
                               test_scaled1, test_scaled2, test_scaled3,  
                               test_target1, test_target2, test_target3
                              )
    
    df_result = make_df_from_estimator(result_dict, iter)
    df_concat = pd.concat([df_estimator, df_result])

    df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)
    
#  4가지 조건이 만족되면 break하고 완료
    val_test = df_concat.loc['acc_val'].iloc[0]
    acc_test = df_concat.loc['acc_test'].iloc[0]
    precision = df_concat.loc['precision'].iloc[0]
    f1score = df_concat.loc['f1score'].iloc[0]
    
    if (val_test >= 0.75 ) & (acc_test > 0.75) & (precision >= 0.8) & (f1score >= 0.6) :
        df_base.to_csv(f'./{com_name}/lgbm_ver{iter}_df_{dt}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_ver{iter}_df_{dt}.pkl')
        break
    if iter >= 1 : 
        df_base.to_csv(f'./{com_name}/lgbm_ver{iter}_df_{dt}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_ver{iter}_df_{dt}.pkl')
        break
    print("******* No.{}  Process is Done! ********".format(iter))
    params_o = make_new_parameter(params_o, df_concat)
    save_to_pickle(f'./{com_name}/params_ver{iter}_{dt}.pkl', params_o)
    
print("**** End of Process ****")
# save model, save df, stoppping 기준 수립
# 일단위로 정확도 측정, 정확도, 정밀도?

# eval_set, early_stopping_rounds 내용 확인 필. ">" 에러 확인.
# 집어 넣으면 에러 발생하니, 원인을 찾아야 함. 하면 좋은 이유는?

# scoring = ['accuracy', 'precision'] 두가지 param 선언이 안되나?
# Precision is ill-defined and being set to 0.0 due to no predicted samples.  왜 나오나 확인해야 함.
# .predict로 하면 positive로 예측하는 경우가 발생하지 않음. 


# feature_importance 확인하고 중요한 feature만 갖고 분석해보기

*** after lgbmgs ******
Fitting 7 folds for each of 256 candidates, totalling 1792 fits
**** End of Process ****


In [224]:
df_base.tail(50)

,value5_x,value5_y
parameter,,
objective,binary,binary
random_state,42,42
reg_alpha,0.0,0.0
reg_lambda,0.0,0.0
silent,warn,warn
subsample,0.7,0.7
subsample_for_bin,200000,200000
subsample_freq,0,0
metric,binary_logloss,binary_logloss


In [125]:
print(lgbmgs.best_estimator_)

LGBMClassifier(colsample_bytree=0.5, learning_rate=0.004,
               metric='binary_logloss', n_estimators=200, num_leaves=6,
               objective='binary', random_state=42, scale_pos_weight=2.5,
               subsample=0.5)


In [56]:
#     num_t = 3
for num_t in range(1, 4, 1):
    test_scaled = globals()[f'test_scaled{num_t}']
    test_target = globals()[f'test_target{num_t}']
    model = lgbmgs.best_estimator_
    accu_all = model.score(test_scaled, test_target)
    y_predict = model.predict(test_scaled)
    precision = precision_score(test_target, y_predict)
    recall = recall_score(test_target, y_predict)
    f1s = f1_score(test_target, y_predict)

    cm = confusion_matrix(test_target, y_predict)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    print(f"----after test : {num_t} ------")
    print(f"accu_all: {accu_all:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1_score: {f1s:.4f}")
    print(f"tn: {tn}, fp: {fp}, fn: {fn}, tp: {tp}")
    print("--------------------------------")

----after test : 1 ------
accu_all: 0.6111, precision: 0.0000, recall: 0.0000, f1_score: 0.0000
tn: 11, fp: 1, fn: 6, tp: 0
--------------------------------
----after test : 2 ------
accu_all: 0.7222, precision: 1.0000, recall: 0.2857, f1_score: 0.4444
tn: 11, fp: 0, fn: 5, tp: 2
--------------------------------
----after test : 3 ------
accu_all: 0.8333, precision: 0.0000, recall: 0.0000, f1_score: 0.0000
tn: 15, fp: 0, fn: 3, tp: 0
--------------------------------


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [126]:
model = lgbmgs.best_estimator_

In [127]:
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [184]:
feature_df.head(40)

,importance
ixic_f_cr,2169.041734
sox_cr,1427.509500
ixic_cr,699.888465
sox_cr_2,674.407957
bank_1,623.178323
privequity_2,602.887969
vix_cr_2,543.902978
corporateetc_1,419.139531
dxy_cr_2,392.885578
low_1,371.468644


In [185]:
new_col = list(feature_df.index[:30]) +  ['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']

In [71]:
new_col

['sox_cr',
 'ixic_f_cr',
 'weekday',
 'vix_cr_2',
 'sox_cr_2',
 'ixic_cr',
 'privequity_2',
 'ixic_cr_2',
 'high_2',
 'corporateetc_1',
 'low_1',
 'kosdaq_cr',
 'dxy_cr_2',
 'dxy_cr',
 'bond_usa_10_cr',
 'kospi_cr',
 'foreigneretc_2',
 'insurance_2',
 'retail_1',
 'foreigner_1',
 'kosdaq_cr_2',
 'dji_cr_2',
 'bond_usa_2_cr',
 'kospi_cr_2',
 'bond_usa_3m_cr_2',
 'vol_2',
 'pension_1',
 'privequity_1',
 'vix_cr',
 'institution_2',
 'cr_00',
 'cr_05',
 'cr_10',
 'cr_15',
 'cr_20']

In [ ]:
model.booster_.feature_importance(importance_type='gain')

In [ ]:
model.booster_.feature_importance(importance_type='split')
# 큰 특징을 가지는 feature는 tree상위레벨에서 적게 사용됨.